In [ ]:
!pip install pillow pytesseract


### 🖼️ 1. Procesamiento automático de horarios desde imágenes con OCR

La función `procesar_horario(...)` permite extraer la información estructurada de un horario escolar a partir de una imagen utilizando OCR (Reconocimiento Óptico de Caracteres) con la biblioteca `pytesseract`.

#### 🧾 Parámetros:
- `imagen_path`: ruta de la imagen del horario.
- `nivel`: grado o nivel académico (por ejemplo, `"7EGB"`).
- `aula`: nombre del paralelo o curso (por ejemplo, `"Reflexivos"`).
- `output_txt`: nombre del archivo `.txt` donde se exportará el resultado.

#### 🔄 Flujo de procesamiento:
1. **OCR:** Se lee el contenido textual de la imagen.
2. **Limpieza:** Se normalizan líneas, se eliminan caracteres no útiles y se convierten a mayúsculas.
3. **Filtrado:** Se excluyen palabras clave asociadas a bloques no académicos (`TUTORIA`, `CLUB`, `RECESO`, etc.).
4. **Correcciones:** Se aplican diccionarios de reemplazo para transformar palabras incompletas o errores de OCR en códigos válidos de materias.
5. **Asignación temporal:** Se asigna cada materia a un día y una franja horaria.
6. **Exportación:** Se guarda un archivo `.txt` con líneas tipo:


In [ ]:
from PIL import Image
import pytesseract

# Si estás en Windows y Tesseract no está en el PATH:
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

def procesar_horario(imagen_path, nivel, aula, output_txt):
    """
    Procesa una imagen de horario y genera un archivo .txt con el formato estandarizado.

    Args:
        imagen_path (str): ruta de la imagen del horario
        nivel (str): grado o nivel académico (ej. '7EGB')
        aula (str): nombre del paralelo o curso (ej. 'Reflexivos')
        output_txt (str): nombre del archivo .txt de salida
    """

    # === Cargar imagen y extraer texto ===
    imagen = Image.open(imagen_path)
    ocr_texto = pytesseract.image_to_string(imagen)

    # === Limpiar líneas ===
    lineas = [
        linea.strip().replace(".", "").replace(",", "").upper()
        for linea in ocr_texto.splitlines()
        if linea.strip()
    ]

    # === Filtrar solo bloques de fondo blanco ===
    exclusiones = {"TUTORIA", "REC", "ALMUERZO", "CLUBES", "CLUB", "EVA", "EVA-LEV"}
    contenido_util = [linea for linea in lineas if all(ex not in linea for ex in exclusiones)]
    contenido_util = contenido_util[:80]

    # === Diccionarios de reemplazo ===
    reemplazos = {
        "PLAS": "PLA", "LEC": "LEC", "SOC": "SOC", "PRO": "PRO", "EDU": "EDU", "ING": "ING", "LEN": "LEN",
        "NAT": "NAT", "MAT": "MAT", "PEN": "PEN", "EXP": "EXP", "CAL": "CAL", "RAZ": "RAZ", "SPE": "SPE",
        "SALUD": "SAL", "ECO": "ECO", "DER": "DER", "COM": "COM", "INV": "INV", "DES": "DES", "DAN": "DAN",
        "MUS": "MUS", "ROB": "ROB", "GEO": "GEO", "QUI": "QUI", "EMP": "EMP", "EST": "EST"
    }

    reemplazos_frase = {
        "FIS 1 JOSE L": "EXP", "MAT 10 JAIR": "RAZ", "ING 7 CRIS": "SPE",
        "PRO TLUD": "PRO", "PRO 1 LUD": "PRO",
        "LEC TANT": "LEC", "LEC T AN": "LEC", "LEC TAN": "LEC"
    }

    # === Generar filas ===
    dias = ["Lunes", "Martes", "Miércoles", "Jueves", "Viernes"]
    contador_hora = {dia: 1 for dia in dias}
    filas_txt = []

    for idx, materia in enumerate(contenido_util):
        dia = dias[idx // 16]
        hora = contador_hora[dia]
        contador_hora[dia] += 1

        if materia in reemplazos_frase:
            codigo = reemplazos_frase[materia]
        else:
            sigla = materia.split()[0]
            codigo = reemplazos.get(sigla, sigla)

        filas_txt.append(f"('{nivel}', '{aula}', '{dia}', {hora}, '{codigo}_{nivel}')")

    # === Exportar archivo .txt ===
    with open(output_txt, "w", encoding="utf-8") as f:
        f.write("\n".join(filas_txt))

    print(f"✅ Horario guardado en: {output_txt}")



### 🗂️ 2. Procesamiento masivo de horarios desde imágenes

En este bloque se aplica la función `procesar_horario(...)` para extraer automáticamente los horarios de múltiples aulas a partir de imágenes `.png`. Cada llamada genera un archivo `.txt` que representa el cromosoma correspondiente a ese curso.

#### 📌 Estructura de cada ejecución:
```python
procesar_horario("imagen.png", "Nivel", "Aula", "archivo_salida.txt")


In [ ]:
procesar_horario("8EGB_Diligentes.png", "8EGB", "Diligentes", "8EGB_Diligentes.txt")
procesar_horario("8EGB_Dinámicos.png", "8EGB", "Dinámicos", "8EGB_Dinámicos.txt")
procesar_horario("8EGB_Disciplinados.png", "8EGB", "Disciplinados", "8EGB_Disciplinados.txt")
procesar_horario("9EGB_Altruistas.png", "9EGB", "Altruistas", "9EGB_Altruistas.txt")
procesar_horario("9EGB_Asertivos.png", "9EGB", "Asertivos", "9EGB_Asertivos.txt")
procesar_horario("9EGB_Audaces.png", "9EGB", "Audaces", "9EGB_Audaces.txt")
procesar_horario("9EGB_Auténticos.png", "9EGB", "Auténticos", "9EGB_Auténticos.txt")
procesar_horario("10EGB_Pacifistas.png", "10EGB", "Pacifistas", "10EGB_Pacifistas.txt")
procesar_horario("10EGB_Perspicaces.png", "10EGB", "Perspicaces", "10EGB_Perspicaces.txt")
procesar_horario("10EGB_Positivos.png", "10EGB", "Positivos", "10EGB_Positivos.txt")
procesar_horario("1BGU_Confiables.png", "1BGU", "Confiables", "1BGU_Confiables.txt")
procesar_horario("1BGU_Íntegros.png", "1BGU", "Íntegros", "1BGU_Íntegros.txt")
procesar_horario("1BGU_Leales.png", "1BGU", "Leales", "1BGU_Leales.txt")
procesar_horario("2BGU_Honestos.png", "2BGU", "Honestos", "2BGU_Honestos.txt")
procesar_horario("2BGU_Justos.png", "2BGU", "Justos", "2BGU_Justos.txt")
procesar_horario("2BGU_Proactivos.png", "2BGU", "Proactivos", "2BGU_Proactivos.txt")
procesar_horario("3BGU_Empáticos.png", "3BGU", "Empáticos", "3BGU_Empáticos.txt")
procesar_horario("3BGU_Solidarios.png", "3BGU", "Solidarios", "3BGU_Solidarios.txt")
procesar_horario("3BGU_Vanguardistas.png", "3BGU", "Vanguardistas", "3BGU_Vanguardistas.txt")



✅ Horario guardado en: 8EGB_Dinámicos.txt
✅ Horario guardado en: 8EGB_Disciplinados.txt
✅ Horario guardado en: 9EGB_Altruistas.txt
✅ Horario guardado en: 9EGB_Asertivos.txt
✅ Horario guardado en: 9EGB_Audaces.txt
✅ Horario guardado en: 9EGB_Auténticos.txt
✅ Horario guardado en: 10EGB_Pacifistas.txt
✅ Horario guardado en: 10EGB_Perspicaces.txt
✅ Horario guardado en: 10EGB_Positivos.txt
✅ Horario guardado en: 1BGU_Confiables.txt
✅ Horario guardado en: 1BGU_Íntegros.txt
✅ Horario guardado en: 1BGU_Leales.txt
✅ Horario guardado en: 2BGU_Honestos.txt
✅ Horario guardado en: 2BGU_Justos.txt
✅ Horario guardado en: 2BGU_Proactivos.txt
✅ Horario guardado en: 3BGU_Empáticos.txt
✅ Horario guardado en: 3BGU_Solidarios.txt
✅ Horario guardado en: 3BGU_Vanguardistas.txt
